# Rumination prediction with cesium

Features are calculated for each band from cwt separately.
Different strategies of ICA and PCA application:

- ICA + PCA on all kind of features (mean, std itd) -> do not increase effectivness of the model

- ICA + PCA on two the best kind of features (std + abs_diff) -> increase effectivness of the model a lot. Second the best method:  

    - 9 * (3 from 2*14) = 27 components
    
            MAPE: 17.9079258661464
            MAE: 0.519917817723914
            MSE: 0.478030015754372
            R^2: 0.376125673763408

    - 9 * (2 from 2*14) = 18 components

            MAPE: 16.8443619132606
            MAE: 0.502078488977514
            MSE: 0.454436875307531
            R^2: 0.406916950702055



- ICA + PCA separately on std features and abs_diffs features -> ok, but not the best way
    
    - 6*(5+5) = 60 components

            MAPE: 24.018846090150365
            MAE: 0.6884866303220416
            MSE: 0.7185622829832321
            R^2: 0.062208343867806604

    - 6*(3+3) = 36 components

            MAPE: 21.50178345656211
            MAE: 0.6201825754057195
            MSE: 0.6069731329548679
            R^2: 0.2078427255904629

    - 6*(2+2) = 24 components

            19.438263116024583
            0.5637319078555214
            0.5362710537170331
            0.3001156176565023

    - 6*(1+1) = 12 components

            19.85447763226303
            0.5760456075230935
            0.585240921194229
            0.23620531480654705

    - 5*(3+3) = 30 components

            MAPE: 20.536346943303492
            MAE: 0.5910318538690427
            MSE: 0.5747574611935253
            R^2: 0.24988722039619093

    - 5*(2+2) = 20 components

            19.35858512090129
            0.5635286480139711
            0.5433040170537524
            0.2909369361542158
            
    - 5*(1+1) = 10 components
    
            20.250558073766026
            0.5886032629783092
            0.5852655385369406
            0.23617318684890465

    - 4*(4+4) = 32 components

            MAPE: 22.031474920258777
            MAE: 0.6380830691061724
            MSE: 0.6277453555688914
            R^2: 0.1807330128932182
            
   
- PCA on flattened ICA channels and PCA separately on std features and abs_diffs features -> more research needed
    
    - 30 from 6*(5+5) = 30 components
    
            MAPE: 21.44571192549426
            MAE: 0.6261229886064391
            MSE: 0.6193306483997083
            R^2: 0.19171500061917268
       
    - 30 from 6*(4+4) = 30 components
    
            MAPE: 21.24921683786726
            MAE: 0.615903392719023
            MSE: 0.608353978391557
            R^2: 0.20604059185814527
    
    - 30 from 6*(3+3) = 30 components
    
            MAPE: 21.11155762577906
            MAE: 0.6154703465611149
            MSE: 0.604724357788718
            R^2: 0.21077758960611548
    
    - 30 from 5*(5+5) = 30 components
    
            MAPE: 21.67828659174979
            MAE: 0.6323799032109819
            MSE: 0.6303037217274262
            R^2: 0.17739410338791517
    
    - 30 from 5*(4+4) = 30 components
    
            MAPE: 21.317971705648553
            MAE: 0.615749388609156
            MSE: 0.6219822296997861
            R^2: 0.1882544365488662
    
    - 30 from 5*(3+3) = 30 components
    
            MAPE: 21.220223056606272
            MAE: 0.6138419950553302
            MSE: 0.6099786827888294
            R^2: 0.20392019914685844
    
    - 30 from 4*(4+4) = 30 components
    
            MAPE; 22.469724109237735
            MAE: 0.6509080551097475
            MSE: 0.6534929953956176
            R^2: 0.14712991074547543
   
   
- PCA on flattened ICA channels and (std + abs_diff) feature sets -> **the best results:**
    
        Example:
        
        
     -  PCA: 18; ICA: 9:
     
             MAPE: 16.1051630051677
             MAE: 0.481721094094576
             MSE: 0.412577754295216
             R^2: 0.461547057719921
             
             
     - PCA: 18; ICA: 18:
     
             MAPE: 11.6348912814115
             MAE: 0.35025761618236
             MSE: 0.237247454583717
             R^2: 0.690369660896321



### Imports

In [ ]:
%load_ext lab_black
import os
import pickle
from time import time
import pywt
import mne
import scipy
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import cesium
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, FloatSlider, interact
from sklearn.decomposition import FastICA
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA


from utils import *

### Loading data

Loading EEG data and data from rumination questionnaire. By default create_df_data load all info from given file but ones can specify it passing list of desired labels from csv file.

In [ ]:
tmin, tmax = -0.1, 0.6
signal_frequency = 256
ERROR = 0
CORRECT = 1

In [ ]:
def load_epochs_from_file(file, reject_bad_segments="auto", mask=None):
    """Load epochs from a header file.

    Args:
        file: path to a header file (.vhdr)
        reject_bad_segments: 'auto' | 'annot' | 'peak-to-peak'

        Whether the epochs with overlapping bad segments are rejected by default.

        'auto' means that bad segments are rejected automatically.
        'annot' rejection based on annotations and reject only channels annotated in .vmrk file as
        'bad'.
        'peak-to-peak' rejection based on peak-to-peak amplitude of channels.

        Rejected with 'annot' and 'amplitude' channels are zeroed.

    Returns:
        mne Epochs

    """
    # Import the BrainVision data into an MNE Raw object
    raw = mne.io.read_raw_brainvision("../data/" + file)

    # Construct annotation filename
    annot_file = file[:-4] + "vmrk"

    # Read in the event information as MNE annotations
    annotations = mne.read_annotations("../data/" + annot_file)

    # Add the annotations to our raw object so we can use them with the data
    raw.set_annotations(annotations)

    # Map with response markers only
    event_dict = {
        "Stimulus/RE*ex*1_n*1_c_1*R*FB": 10004,
        "Stimulus/RE*ex*1_n*1_c_1*R*FG": 10005,
        "Stimulus/RE*ex*1_n*1_c_2*R": 10006,
        "Stimulus/RE*ex*1_n*2_c_1*R": 10007,
        "Stimulus/RE*ex*2_n*1_c_1*R": 10008,
        "Stimulus/RE*ex*2_n*2_c_1*R*FB": 10009,
        "Stimulus/RE*ex*2_n*2_c_1*R*FG": 10010,
        "Stimulus/RE*ex*2_n*2_c_2*R": 10011,
    }

    # Map for merged correct/error response markers
    merged_event_dict = {"correct_response": 0, "error_response": 1}

    # Reconstruct the original events from Raw object
    events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

    # Merge correct/error response events
    merged_events = mne.merge_events(
        events,
        [10004, 10005, 10009, 10010],
        merged_event_dict["correct_response"],
        replace_events=True,
    )
    merged_events = mne.merge_events(
        merged_events,
        [10006, 10007, 10008, 10011],
        merged_event_dict["error_response"],
        replace_events=True,
    )

    epochs = []
    bads = []
    this_reject_by_annotation = True

    if reject_bad_segments != "auto":
        this_reject_by_annotation = False

    # Read epochs
    temp_epochs = mne.Epochs(
        raw=raw,
        events=merged_events,
        event_id=merged_event_dict,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        reject_by_annotation=this_reject_by_annotation,
        preload=True,
    )

    if reject_bad_segments == "annot":
        custom_annotations = get_annotations(annot_file)
        bads = get_bads_by_annotation(custom_annotations)
    elif reject_bad_segments == "peak-to-peak":
        bads = get_bads_by_peak_to_peak_amplitude(temp_epochs)
    else:
        epochs = temp_epochs
        return epochs

    if mask is None:
        epochs = clear_bads(temp_epochs, bads)
    elif len(mask) == 64:
        epochs = reject_with_mask(temp_epochs, mask, bads)
    else:
        print(
            "Given mask has wrong shape. Expected len of 64 but got {}".format(
                len(mask)
            )
        )

    return epochs

In [ ]:
def create_df_data(
    test_participants=False,
    test_epochs=False,
    info_filename=None,
    info=["Rumination Full Scale"],
):
    """Loads data for all participants and create DataFrame with optional additional info from given .csv file.
    Participants with less than 10 epochs per condition are rejected.

    Parameters
    ----------
    test_participants: bool
        whether load data for training or final testing.
        If true load participants data for testing.
    test_epochs: bool
        whether load data for training or final testing.
        If true load epochs of each participants data for testing.
    info_filename: String | None
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.


    Returns
    -------
    go_nogo_data_df : pandas.DataFrame

    """
    header_files = glob.glob("../data/responses/*.vhdr")
    header_files = sorted(header_files)
    go_nogo_data_df = pd.DataFrame()

    for file in header_files:
        #  load eeg data for given participant
        participant_epochs = load_epochs_from_file(file)

        # and compute participant's id from file_name
        participant_id = re.match(r".*_(\w+).*", file).group(1)

        error = participant_epochs["error_response"]._data
        correct = participant_epochs["correct_response"]._data

        # exclude those participants who have too few samples
        if len(error) < 10 or len(correct) < 10:
            # not enough data for this participant
            continue

        # construct dataframe for participant with: id|epoch_data|response_type|additional info...
        participant_df = create_df_from_epochs(
            participant_id, correct, error, info_filename, info
        )
        print(participant_id)
        go_nogo_data_df = go_nogo_data_df.append(participant_df, ignore_index=True)

    return go_nogo_data_df

In [ ]:
def create_df_from_epochs(id, correct, error, info_filename, info):
    """Create df for each participant. DF structure is like: {id: String ; epoch: epoch_data ; marker: 1.0|0.0}
    1.0 means correct and 0.0 means error response.
    Default info extracted form .csv file is 'Rumination Full Scale' and participants' ids.
    With this info df structure is like:
    {id: String ; epoch: epoch_data ; marker: 1.0|0.0 ; File: id ; 'Rumination Full Scale': int}

    Parameters
    ----------
    id: String
        participant's id extracted from filename
    correct: array
        correct responses' data
    error: array
        error responses' data
    info_filename: String
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.

    Returns
    -------
    participant_df : pandas.DataFrame

    """
    participant_df = pd.DataFrame()
    info_df = pd.DataFrame()

    # get additional info from file
    if info_filename is not None:
        rumination_df = pd.read_csv(info_filename, usecols=["File"] + info)
        info_df = (
            rumination_df.loc[rumination_df["File"] == id]
            .reset_index()
            .drop("index", axis=1)
        )

    for epoch in correct:
        epoch_df = pd.DataFrame(
            {"id": [id], "epoch": [epoch], "marker": [CORRECT]}
        ).join(info_df)
        participant_df = participant_df.append(epoch_df, ignore_index=True)

    for epoch in error:
        epoch_df = pd.DataFrame({"id": [id], "epoch": [epoch], "marker": [ERROR]}).join(
            info_df
        )
        participant_df = participant_df.append(epoch_df, ignore_index=True)

    return participant_df

In [ ]:
df_name = "go_nogo_df"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df = create_df_data(info_filename=info_filename)
    epochs_df.name = df_name
    # save loaded data into a pickle file
    epochs_df.to_pickle("../data/" + epochs_df.name + ".pkl")
    print("Done. Pickle file created")

Data is now read into dataframe and each epoch is a single record.

Sorting participants by the number of errors, descending. This way the best participants are first.

In [ ]:
# add new columns with info about error/correct responses amount
grouped_df = epochs_df.groupby("id")
epochs_df["error_sum"] = grouped_df[["marker"]].transform(
    lambda x: (x.values == ERROR).sum()
)
epochs_df["correct_sum"] = grouped_df[["marker"]].transform(
    lambda x: (x.values == CORRECT).sum()
)

# mergesort for stable sorting
epochs_df = epochs_df.sort_values("error_sum", ascending=False, kind="mergesort")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from tempfile import mkdtemp


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


import numpy as np
import scipy.stats

## Training and predictions

- Computes ICA and then at each channel computes CWT (ica_n_components = N).
- For each band (frequency) from CWT set it computes features given in feature_dict parameter (eg. std or mean).
- Then it computes PCA on flattened ICA channels and features (outer_components = N)
- Ending feature vector has shape: outer_components from (ica_n_components * len(feature_dict) * frequencies)

In [ ]:
def std_signal(t, m, e):
    return np.std(m)


def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))

In [ ]:
guo_features = {
    "std": std_signal,
    "abs_diffs": abs_diffs_signal,
}

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    ## Note: does not handle mix 1d representation
    # if _is_1d(y_true):
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def r2_adjusted_scorer(y_test, y_pred, p, n):
    r2 = r2_score(y_test, y_pred)
    r2_adj = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    return r2_adj

### Regressions grid search

**Warning! It takes a lot of time!** 

It is a pipeline which allows manipulation of vectorization's parameters. Base steps dictionary consists all steps of vectorization including standarization of data. 

In rate_regression function, using GridSearchCV, cross-validation splitting strategy can be specified. Default cv = 5.
Results of cross-validated search are in **grid_search.cv_results** and chosen model is in **grid_search.best_estimator_**

In [ ]:
X = np.array(epochs_df[epochs_df["marker"] == ERROR]["epoch"].to_list())
y = np.array(epochs_df[epochs_df["marker"] == ERROR]["Rumination Full Scale"].to_list())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
def IcaPreprocessingTransformer():
    def transform(X):
        timepoints_per_channel = np.concatenate(X, axis=1)
        return timepoints_per_channel.T

    return FunctionTransformer(func=transform)


def CwtVectorizer(timepoints_count, feature_dict, mwt="morl", cwt_density=2):
    def transform(X):
        X_ica_transposed = X.T
        ica_n_components = X.shape[1]

        epochs_count = int(X_ica_transposed.shape[1] / timepoints_count)
        data_per_channel = X_ica_transposed.reshape(
            ica_n_components, epochs_count, timepoints_count
        )

        vectorized_data = []
        for data in data_per_channel:
            data_cwt = np.array([cwt(epoch, mwt, cwt_density) for epoch in data])

            # cesium functions
            feature_set_cwt = featurize.featurize_time_series(
                times=None,
                values=data_cwt,
                errors=None,
                features_to_use=list(feature_dict.keys()),
                custom_functions=feature_dict,
            )
            features_per_epoch = feature_set_cwt.to_numpy()
            vectorized_data.append(features_per_epoch)

        vectorized_data = np.array(vectorized_data)
        vectorized_data = np.stack(vectorized_data, axis=1)
        epochs_per_channel_feature = vectorized_data.reshape(
            vectorized_data.shape[0], -1
        )
        return epochs_per_channel_feature

    return FunctionTransformer(func=transform)

In [ ]:
base_steps = [
    ("ica_preprocessing", IcaPreprocessingTransformer()),
    ("ica", FastICA(random_state=5)),
    (
        "cwt",
        CwtVectorizer(
            timepoints_count=X.shape[-1],
            feature_dict=guo_features,
        ),
    ),
    ("pca", PCA(random_state=5)),
    ("scaler", StandardScaler()),
]

In [ ]:
def rate_regression(
    X_train, y_train, X_test, y_test, regressor, regressor_params, cv=5
):
    #     cachedir = mkdtemp()
    pipeline = Pipeline(steps=base_steps + [regressor])
    param_grid = regressor_params
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv,
        scoring={
            "r2": "r2",
            "mae": "neg_mean_absolute_error",
        },
        refit="r2",
        n_jobs=3,
        verbose=10,
    )
    grid_search.fit(X_train, y_train)
    predictions = grid_search.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = grid_search.score(X_test, y_test)
    r2_adj = r2_adjusted_scorer(y_test, predictions, len(X_test[0]), len(X_test))
    print(
        f"Best result: MAPE: {mape} MAE: {mae} MSE: {mse} R^2: {r2} R^2 adjusted: {r2_adj}"
    )
    return grid_search

#### Prediction with SVR

In [ ]:
svr = ("svr", SVR(kernel="rbf"))
regressor_params = dict(
    ica__n_components=np.arange(35, 36, 1),
    pca__n_components=np.arange(20, 21, 1),
    svr__C=np.arange(1, 2, 1),
    svr__gamma=[0.1],
    svr__epsilon=[0.1],
)
svr_grid_search_df = rate_regression(
    X_train, y_train, X_test, y_test, svr, regressor_params
)

Dummy Classifier for baseline:

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)

y_pred = dummy_regr.predict(X_test)
print(mean_absolute_percentage_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(model.score(X_test, y_test))